<a href="https://colab.research.google.com/github/JasmineAdvanture/fatty-liver-project/blob/dev-preproc/Fatty_Liver_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [16]:
#Imports 
import pandas as pd
import numpy as np

In [18]:
#read dataset as df
df = pd.read_excel("NTCMRC_all.xlsx")
df.head(5)


,sid,P_Number,birth,data_exam,CMRC_id,sex,age,data_year,year_come,height_y,weight_y,waist_y,systolic_y,diastolic_y,heartbeat_y,arm_circumference,calf_circumference,Thigh_circumference,Glucose_AC_y,Triglyceride_y,HDL_C_y,WBCb,RBCb,Hemoglobin,Hematocrit,MCV,MCH,MCHC,RDW,Platelets,Transferrin,Ferritin,Folate,AST_GOT,ALT_GPT,ALK_P,Total_Bilirubin,D_Bilirubin,gamgt,AFP,Insulin,T_Cholesterol,LDL_C_direct,VLDL_C,Non_HDL_C,T_CHOL_HDL_C,LDL_C_HDL_C,HS_CRP,BUN,Creatinine,Estimated_GFR_x,sodium,K,Cl,CO2,Calcium,Inorganic_P,Albumin,Total_Protein,Hb_A1c,Estimated_AG,Intact_PTH,Uric_Acid,HBsAg_x,RPR,RF,H_PYAB_x,Anti_HCV_x,Color,Turbidity,SP_Gravity,pH,Leukocyte,Nitrite,Protein,Glucose,Ketone,Urobilinogen,Bilirubin,octBlood,RBCu,WBCu,Squamus_Epi,UreaNU,MicroALB_U,Creatinine_U,TP_U,Alb_Cre_ratio,TP_U_Cre_ratio,HOMA_IR,Insulin_GLU,Adiponectin,Leptin,TNF_A,TotalVitaminD,Vitamin_B12,Lymphocyte,Monocyte,Eosinophil,Basophil,Segment,bloodtype,bloodtype_rh,diet,diet_1,education,job,job_1,job_t,job_c,job_q,job_qt,religion,religion_2,f_clan,f_land,f_land1,m_clan,m_land,m_land1,mstatus,marriyr,children,fam_self,fam_p,fam_sp,fam_bs,fam_chi,fam_lawchi,fam_grandchi,fam_o,child_num,grandchild,others,live_place1,live_place2,year1,year2,selfcare,care_self,care_p,care_sp,care_bs,care_chi,care_lawchi,care_n,care_o,caregiver_1,eco_self,eco_p,eco_sp,eco_bs,eco_chi,eco_lawchi,eco_gov,eco_f,eco_o,ecosource_1,insurance,insurance_1,screening,screening_o,together,together_1,together_2,smoke,s_smoke,smoke_t,smoke_q,smoke_qt1,smoke_qt2,smoke_f,smoke_n,smoke_second,tea,tea_c,tea_1,tea_t,tea_q,tea_f,tea_v,coffee,coffee_c,coffee_t,coffee_q,coffee_f,coffee_v,betel,s_betel,betel_clt1,betel_clt2,betel_clt3,betel_clt4,betel_clt5,betel_clt6,betel_o,betel_t,betel_q,betel_f,betel_n,activity_t,carryh,sport,sport_1,sport_d,sport_t,cardio,hypertension,Dysrhythmia,ap,ami,Hyperlipidemia,HF,f_cardio,f_hyper,f_dys,f_ap,f_ami,f_lipid,f_hf,m_cardio,m_hyper,m_dys,m_ap,m_ami,m_lipid,m_hf,bs_cardio,bs_hyper,bs_hyper_1,bs_dys,bs_dys_1,bs_ap,bs_ap1,bs_ami,bs_ami_1,bs_lipid,bs_lipid_1,bs_hf,bs_hf_1,chi_cardio,chi_hyper,chi_dys,chi_ap,chi_ami,chi_lipid,chi_hf,endocrine,diabetes,Thyroid,f_dm,f_Thyroid,m_dm,m_Thyroid,bs_dm,bs_dm_1,bs_Thyroid,bs_Thyroid_1,chi_dm,chi_Thyroid,pepticulcer,gastritis,hbv,hcv,hepatitis_o,FLD,fibrosis,Cirrhosis,Polyposis,ibs,f_pud,f_hbv,f_hcv,f_liver,f_fld,f_f,f_lc,f_polyp,f_ibs,m_pud,m_hbv,m_hcv,m_liver,m_fld,m_f,m_lc,m_polyp,m_ibs,bs_pud,bs_pud_1,bs_hbv,bs_hbv_1,bs_hcv,bs_hcv_1,bs_liver,bs_fld,bs_fld_1,bs_f,bs_f_1,bs_lc,bs_lc_1,bs_polyp,bs_polyp_1,bs_ibs,bs_ibs_1,chi_pud,chi_hbv,chi_hcv,chi_liver,chi_fld,chi_f,chi_lc,chi_polyp,chi_ibs,respiratory,tb,Asthma,apnea,copd,f_resp,f_Asthma,f_apnea,f_copd,m_resp,m_Asthma,m_apnea,m_copd,bs_resp,bs_Asthma,bs_Asthma_1,bs_apnea,bs_apnea_1,bs_copd,bs_copd_1,chi_resp,chi_Asthma,chi_apnea,chi_copd,anemia,Hemophilia,f_anemia,f_h,m_anemia,m_h,bs_anemia,bs_anemia_1,bs_h,bs_h_1,chi_anemia,chi_h,stroke,brainbleed,head_injury,epilepsy,Parkinsons,dementia,Huntingtons,f_stroke,f_hi,f_brainbleed,f_epilep,f_pd,f_demen,f_hd,m_stroke,m_hi,m_brainbleed,m_epilep,m_pd,m_demen,m_hd,bs_stroke,bs_stroke_1,bs_hi,bs_hi_1,bs_brainbleed,bs_brainbleed_1,bs_epilep,bs_epilep_1,bs_pd,bs_pd_1,bs_demen,bs_demen_1,bs_hd,bs_hd_1,chi_stroke,chi_hi,chi_brainbleed,chi_epilep,chi_pd,chi_demen,chi_hd,insomnia,Depression,mdd_dx,mental,f_mdd,f_mdd_dx,f_mental,m_mdd,m_mdd_dx,m_mental,bs_mdd,bs_mdd_1,bs_mdd_dx,bs_mental,chi_mdd,chi_mdd_dx,chi_mental,re_1,ckd,dialysis,gout,stone,bpn,f_re,f_ckd,f_dialysis,f_stone,f_bpn,m_re,m_ckd,m_dialysis,m_stone,m_bpn,bs_re,bs_ckd,bs_ckd_1,bs_dialysis,bs_dialysis_1,bs_stone,bs_stone_1,bs_bpn,bs_bpn_1,chi_re,chi_ckd,chi_dialysis,chi_stone,chi_bpn,cataract,glau,retina,sicca,f_cata,f_glau,f_retina,f_sicca,m_cata,m_glau,m_retina,m_sicca,bs_cata,bs_cata_1,bs_glau,bs_glau_1,bs_retina,bs_retina_1,bs_sicca,bs_sicca_1,chi_cata,chi_glau,chi_retina,chi_sicca,cancer,cancer_1,cantime,cancer_mi,lungcan,breastcna,colican,gastrican,livercan,cancer_o,f_c

# 新段落

Data Type Conversion

In [20]:
# Create a copy of df as df1
df1 = df.copy()

# Specify the columns to be converted to float
columns_to_convert1 = ['BMI', 'Triglyceride_y', 'gamgt', 'waist_y', 'mst', 'egfrn', 'Estimated_GFR_x', 'Alb_Cre_ratio', 'HOMA_IR', 'HS_CRP']

# Specify the columns to be converted to int
columns_to_convert2 = ['脂肪肝 fatty Liver (0:正常  1:mild 2:moderate 3:severe)']


# Convert the specified columns to float and fill missing/unconvertible values with NaN
for column in columns_to_convert1:
    df1[column] = pd.to_numeric(df1[column], errors='coerce')
    
# Convert the specified columns to int and fill missing/unconvertible values with NaN
for column in columns_to_convert2:
    df1[column] = pd.to_numeric(df1[column], errors='coerce').astype(float).astype(pd.Int64Dtype())

In [21]:
#Data Type Checks
column_list = ['脂肪肝 fatty Liver (0:正常  1:mild 2:moderate 3:severe)', 'BMI', 'Triglyceride_y', 'gamgt', 'waist_y', 'mst', 'egfrn', 'Estimated_GFR_x', 'Alb_Cre_ratio', 'HOMA_IR', 'HS_CRP']
data_types = df1[column_list].dtypes
print(data_types)


脂肪肝 fatty Liver (0:正常  1:mild 2:moderate 3:severe)      Int64
BMI                                                   float64
Triglyceride_y                                        float64
gamgt                                                 float64
waist_y                                               float64
mst                                                     int64
egfrn                                                 float64
Estimated_GFR_x                                       float64
Alb_Cre_ratio                                         float64
HOMA_IR                                               float64
HS_CRP                                                float64
dtype: object


Derive Target Variables

In [22]:
# Calculate FLI using the formula and defined as df2
df2 = df1.copy()
df2['FLI'] = (np.exp(0.953 * np.log(df2['Triglyceride_y']) + 0.139 * df2['BMI'] + 0.718 * np.log(df2['gamgt']) \
     + 0.053 * df2['waist_y'] - 15.745)) / (1 + np.exp(0.953 * np.log(df2['Triglyceride_y']) \
    + 0.139 * df2['BMI'] + 0.718 * np.log(df2['gamgt']) + 0.053 * df2['waist_y'] - 15.745)) * 100

# Derive FL_echo based on ultrasound results column
df2['FL_echo'] = df2['脂肪肝 fatty Liver (0:正常  1:mild 2:moderate 3:severe)']
df2['FL_echo'] = df2['FL_echo'].replace('<NA>', np.nan)

#Derive fl_status column based on fl_echo and FLI(when available): 1 for positive; 0 for negative; -1 for unavailable
def derive_fl_status(row):
    # Derive FL_Check column to infer the status by echo or FLI
    liver_status = row['脂肪肝 fatty Liver (0:正常  1:mild 2:moderate 3:severe)']
    fli_value = row['FLI']
    
    if pd.isna(liver_status) and pd.isna(fli_value):
        return -1
    elif pd.notna(liver_status) and liver_status != 0:
        return 1
    elif pd.notna(fli_value) and fli_value >= 60:
        return 1
    else:
        return 0

df2['fl_status'] = df2.apply(derive_fl_status, axis=1)

#Derive homa_ir_check, hs_crp_check, and mst_total to determine MAFLD risk factors
df2['homa_ir_check'] = df2['HOMA_IR'].apply(lambda x: 1 if x >= 2.5 else 0)
df2['hs_crp_check'] = df2['HS_CRP'].apply(lambda x: 1 if x > 2 else 0)
df2['mst_total'] = df2[['w', 'hyper', 'HDL', 'fg', 'trig', 'homa_ir_check', 'hs_crp_check']].sum(axis=1)

In [23]:
#Derive target variables and named as df3
def derive_MAFLD(df):
    df['MAFLD'] = 0  # Initialize MAFLD field as 0

    # Condition 1: fl_status = -1
    df.loc[df['fl_status'] == -1, 'MAFLD'] = -1

    # Condition 2: fl_check = 0
    df.loc[df['fl_status'] == 0, 'MAFLD'] = 0

    # Condition 3: fl_check = 1
    # Subcondition 1: BMI >= 23
    df.loc[(df['fl_status'] == 1) & (df['BMI'] >= 23), 'MAFLD'] = 1

    # Subcondition 2: BMI < 23 and mst >= 2
    df.loc[(df['fl_status'] == 1) & (df['BMI'] < 23) & (df['mst_total'] >= 2), 'MAFLD'] = 1

    # Subcondition 3: DM_determine = 1
    df.loc[(df['fl_status'] == 1) & (df['DM_determine'] == 1), 'MAFLD'] = 1

    return df

def derive_CKD(df):
    # Initialize CKD field as -1
    df['CKD'] = -1

    # Condition 1: egfrn >= 60 and Alb_Cre_ratio < 3
    df.loc[(df['Estimated_GFR_x'] >= 60) & (df['Alb_Cre_ratio'] < 3), 'CKD'] = 1

    # Condition 2: egfrn >= 60 and 3 <= Alb_Cre_ratio <= 30 or 45 <= egfrn < 60 and Alb_Cre_ratio < 3
    df.loc[((df['Estimated_GFR_x'] >= 60) & (df['Alb_Cre_ratio'].between(3, 30))) |
           ((df['Estimated_GFR_x'].between(45, 60)) & (df['Alb_Cre_ratio'] < 3)), 'CKD'] = 2

    # Condition 3: egfrn >= 60 and Alb_Cre_ratio > 30 or egfrn < 60 and Alb_Cre_ratio >= 0
    df.loc[((df['Estimated_GFR_x'] >= 60) & (df['Alb_Cre_ratio'] > 30)) |
           ((df['Estimated_GFR_x'] < 60) & (df['Alb_Cre_ratio'] >= 0)), 'CKD'] = 3

    # Set CKD as 0 for cases where egfrn and Alb_Cre_ratio are not empty and CKD is still -1
    df.loc[(df['Estimated_GFR_x'].notnull()) & (df['Alb_Cre_ratio'].notnull()) & (df['CKD'] == -1), 'CKD'] = 0

    return df

df3 = derive_CKD(derive_MAFLD(df2))

Derive Patient MAFLD Validity Status in complete, partial and unvailable

In [24]:
# Derive FL_group_list, 对CMRC_id分组并计算每个病人的FL_Check的唯一值
grouped = df2.groupby('CMRC_id')['fl_status'].unique()
df2['fl_group_list'] = df2.groupby('CMRC_id')['fl_status'].transform(lambda x: [x.unique().tolist()] * len(x))

# Derive patient_fl_validity according to FL_group_list conditions
def assign_patient_fl_validity(df):
    df['patient_fl_validity'] = -1  # 初始化所有记录为第三组 (-1)
    
    def get_patient_valid(group_list):
        if isinstance(group_list, list):
            if -1 in group_list and len(group_list) == 1:
                return "unavailable"  # 第三组 (-1)
            elif -1 in group_list:
                return "partial"  # 第二组 (0)
            else:
                return "completed"  # 第一组 (1)
        else:
            return "others"  # 第三组 (-1)
    
    df['patient_fl_validity'] = df['fl_group_list'].apply(get_patient_valid)
    
    return df


#assign patient valid value
df4 = assign_patient_fl_validity(df3)

In [25]:
# Checks for patient fatty liver validity status
df4['patient_fl_validity'].value_counts()

completed      21749
partial          602
unavailable      483
Name: patient_fl_validity, dtype: int64

Grouping data in the patient level and using sliding window for predictions

In [26]:
def sliding_window_data(df, input_window_size, target_window_size):
    transformed_data = []
    group_counter = {}
    
    df_sorted = df.sort_values(['CMRC_id', 'year_come'])
        
    for patient_id, group in df_sorted.groupby('CMRC_id'):
        if len(group) < input_window_size + target_window_size:
            continue

        
        group_counter.setdefault(patient_id, 0)
        group_counter[patient_id] += 1
        group_alias = f'{patient_id}_group{group_counter[patient_id]}'
    
        for i in range(len(group) - input_window_size - target_window_size + 1):
            input_data = group[i:i+input_window_size]
            target_data = group[i+input_window_size:i+input_window_size+target_window_size]

            # Flatten input_data and repeat target_data
            input_features_t1 = input_data.iloc[0, :].values.flatten()
            input_features_t2 = input_data.iloc[1, :].values.flatten()
            t3_MAFLD = target_data['MAFLD'].values

            new_row = [group_alias] + list(input_features_t1) + list(input_features_t2) + list(t3_MAFLD)

            transformed_data.append(new_row)

    columns_list = ['CMRC_id'] + [f't1_{col}' for col in input_data.columns] + [f't2_{col}' for col in input_data.columns] + [f't3_MAFLD']
    transformed_df = pd.DataFrame(transformed_data, columns=columns_list)
    return transformed_df


# Sliding window implementation, ie. use previous 2 years record to predict the 3 year MAFLD status
df5 = sliding_window_data(df4, input_window_size=2, target_window_size=1)


In [27]:
pd.set_option('display.max_columns', None)
df5.head(5)

CMRC_id       t1_sid t1_P_Number  t1_birth  t1_data_exam  \
0  A02013O51207P5052_group1  20131207052     P020859    430127      20131207   
1  A02013O51207P5052_group1  20141220048     P032306    430127      20141220   
2  A02013O51207P5052_group1  20160409051     P050743    430127      20160409   
3  A02013O51207P5052_group1  20170429016     P060991    430127      20170429   
4  A02014I70719D9059_group1  20140719059     P031204    450628      20140719   

          t1_CMRC_id  t1_sex  t1_age  t1_data_year  t1_year_come t1_height_y  \
0  A02013O51207P5052       0      59          2013             1         148   
1  A02013O51207P5052       0      60          2014             2         149   
2  A02013O51207P5052       0      62          2016             3         147   
3  A02013O51207P5052       0      63          2017             4         148   
4  A02014I70719D9059       0      58          2014             1         146   

  t1_weight_y  t1_waist_y t1_systolic_y t1_diastolic_y t1_heartbeat_y  \
0          70        82.0           134             81             89   
1          71        83.0           141             74             82   
2        71.3        85.0           116             67             82   
3        71.9        84.0           131             77             88   
4          44        61.0           149             96             91   

  t1_arm_circumference t1_calf_circumference t1_Thigh_circumference  \
0                   \N                    \N                     \N   
1                   \N                    \N                     \N   
2                   \N                    \N                     \N   
3                   \N                    \N                     \N   
4                   \N                    \N                     \N   

  t1_Glucose_AC_y  t1_Triglyceride_y t1_HDL_C_y t1_WBCb t1_RBCb t1_Hemoglobin  \
0             101              264.0       56.4     5.8    4.48          13.7   
1             104              255.0       55.8       5    4.44          13.4   
2             103              192.0       60.7     6.2    4.63          14.1   
3             111              164.0         83     5.4    4.36          13.1   
4              99               75.0       60.4     4.2    4.87          13.8   

  t1_Hematocrit t1_MCV t1_MCH t1_MCHC t1_RDW t1_Platelets t1_Transferrin  \
0          41.2     92   30.6    33.3   12.7          191          242.9   
1          40.3   90.8   30.2    33.3   12.4          190             \N   
2          42.9   92.7   30.5    32.9   12.8          193            262   
3          39.9   91.5     30    32.8   12.7          218             \N   
4          42.6   87.5   28.3    32.4   12.8          268          240.9   

  t1_Ferritin t1_Folate t1_AST_GOT t1_ALT_GPT t1_ALK_P t1_Total_Bilirubin  \
0        52.1     14.16         21         19       57                0.7   
1         NaN       NaN         15         18       55                0.5   
2        62.1     12.41         18         22       50                0.7   
3         NaN       NaN         27         30       46                0.4   
4        94.6     19.34         23         17       90                0.8   

  t1_D_Bilirubin  t1_gamgt t1_AFP t1_Insulin t1_T_Cholesterol t1_LDL_C_direct  \
0            0.1      15.0    2.8       9.18              315           196.5   
1            0.1      14.0    2.6       5.86              312             199   
2          < 0.1      17.0    4.1       4.56              213             115   
3            0.1      16.0    3.9        6.3              267           157.6   
4            0.1      10.0    3.3       3.75              198           117.8   

  t1_VLDL_C t1_Non_HDL_C t1_T_CHOL_HDL_C t1_LDL_C_HDL_C  t1_HS_CRP t1_BUN  \
0        53          259            5.59           3.48      1.347     15   
1        51          256            5.59           3.57      2.252     14   
2        38          152            3.51           1.89      0.532 

In [28]:
#Checks
filtered_data = df5[(df5['t1_MAFLD'] != -1) & (df5['t2_MAFLD'] != -1) & (df5['t3_MAFLD'] != -1)]
value_counts = filtered_data['t2_patient_fl_validity'].value_counts()
print(value_counts)

completed    9386
partial       168
Name: t2_patient_fl_validity, dtype: int64
